In [1]:
import numpy as np
import itertools
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import pandas as pd

In [2]:
class datarec:
    
    def __init__(self, dataframe, constraint_matrix_file):
        
        self.df = dataframe.copy()
        
        self.cm = pd.read_excel(constraint_matrix_file)
        
    def objfun(self, x):
        
        return np.transpose(self.xraw - x) @ self.Vinv @ (self.xraw - x)
        
    def reconcile(self):
        
        y_raw = np.array(self.df)
        
        varvec = np.var(y_raw, axis=0)
        
        V = np.diag(varvec.reshape(-1))
        
        self.Vinv = np.linalg.pinv(V)
        
        A = np.array(self.cm)
        
        B = np.eye(len(self.cm.columns))
        
        cons = ({'type': 'eq', 'fun': lambda x: A @ x},{'type': 'ineq', 'fun': lambda x: B @ x})
        
        y_rec = np.zeros(y_raw.shape)
        
        for k in range(y_raw.shape[0]):
            
            self.xraw = y_raw[k, :]
                
            rec = minimize(self.objfun, self.xraw, constraints=cons, method='SLSQP', options={"maxiter": 200, 'disp': False})
            
            y_rec[k, :] = rec.x
            
        df1 = self.df.copy()
        
        df1.iloc[:, :] = y_rec
        
        return df1

In [3]:
F0 = 100
F = np.zeros(6)
F[0] = F0
F[1] = 0.5*F0
F[2] = 0.5*F0
F[3] = (1-0.5)*F0
F[4] = (1-0.5)*F0
F[5] = F0
#print("True flow rates: ", F)
np.random.seed(1000)
#np.random.seed(1976)

# Noise and bias values
noise_vec = np.random.uniform(0, 3, 6)
#print("Noise (Percentage): ", noise_vec)

bias_vec = np.random.uniform(-5, 5, 6)
#print("Bias (Percentage): ", bias_vec)

# Generate time series data for flow rates with noise and bias added to the true flow rates
N=1000
y_raw = np.zeros((N,6))
for k in np.arange(6):
    bias_val = (bias_vec[k]/100.0)* F[k]
    std_val =  (noise_vec[k]/100.0)* F[k]
    y_raw[:,k] =  F[k] + bias_val + np.random.normal(0, std_val, N)

y_raw

array([[ 93.06187989,  49.60509645,  48.71021595,  51.91601952,
         46.30601332, 102.76050997],
       [ 94.61110225,  49.14468207,  50.04880494,  50.41732393,
         48.65220737, 101.89368599],
       [ 95.04949646,  49.7130259 ,  50.28795792,  52.02696046,
         46.45622759, 102.72272923],
       ...,
       [ 96.6242025 ,  49.50970813,  50.73568014,  51.31582692,
         49.57675002, 102.48104716],
       [ 94.31311615,  49.64167492,  50.9935969 ,  53.0907262 ,
         48.95011856, 102.58880307],
       [ 91.54020944,  49.69687931,  45.94910724,  51.87512861,
         47.26368085, 101.59209244]])

In [4]:
df=pd.DataFrame(y_raw)

In [5]:
df.columns = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6']

In [6]:
df

,F1,F2,F3,F4,F5,F6
0,93.061880,49.605096,48.710216,51.916020,46.306013,102.760510
1,94.611102,49.144682,50.048805,50.417324,48.652207,101.893686
2,95.049496,49.713026,50.287958,52.026960,46.456228,102.722729
3,95.609434,49.487267,48.854575,52.838192,48.224764,102.351741
4,95.135683,49.607312,49.916982,51.109979,47.361711,101.255035
...,...,...,...,...,...,...
995,91.774823,49.501328,47.822685,51.223204,48.902359,102.956476
996,97.699879,49.525768,48.910608,52.361762,47.189290,102.051975
997,96.624202,49.509708,50.735680,51.315827,49.576750,102.481047
998,94.313116,49.641675,50.993597,53.090726,48.950119,102.588803


In [7]:
o1=datarec(df, 'cooling water constraints.xlsx')

In [8]:
df_rec = o1.reconcile()

In [9]:
df_rec

,F1,F2,F3,F4,F5,F6
0,100.421694,49.838661,50.583033,49.838661,50.583033,100.421694
1,100.380790,49.275425,51.105364,49.275425,51.105364,100.380790
2,100.806568,49.930304,50.876263,49.930304,50.876263,100.806568
3,100.651256,49.753814,50.897442,49.753814,50.897442,100.651256
4,99.928951,49.745807,50.183145,49.745807,50.183145,99.928951
...,...,...,...,...,...,...
995,100.706953,49.683366,51.023587,49.683366,51.023587,100.706953
996,100.457221,49.773769,50.683451,49.773769,50.683451,100.457221
997,101.252850,49.659921,51.592929,49.659921,51.592929,101.252850
998,101.155634,49.881561,51.274073,49.881561,51.274073,101.155634
